In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy

In [2]:
# forward prop
## sigmoid activation function using pytorch
def sigmoid_activation(z):
    return 1 / (1 + torch.exp(-z))

def forward_prop(_data, _nodes, _weights, _biases):
    layer_vals = []
    a = _data
    for i in range(len(_nodes)-2): # feed thru hidden layers
        z = torch.mm(a, _weights[i]) + _biases[i]
        a = sigmoid_activation(z)
        layer_vals.append(a)
    z_out = torch.mm(a, _weights[-1]) + _biases[-1]
    output = sigmoid_activation(z_out)
    layer_vals.append(output)
#     print(output)
    return layer_vals

In [3]:
# loss computation
# loss = y - output
def calculate_loss(_layer_vals):
    return y - _layer_vals[-1]

In [4]:
# back propogation
## function to calculate the derivative of activation
def sigmoid_delta(x):
  return x * (1 - x)

In [5]:
def backprop_and_update(_layer_vals, _weights, _biases, _data):
    loss = calculate_loss(_layer_vals)
    deltas = [] # from first hidden to output
    ds = [] # from output to first hidden
    # compute derivative of error terms
    for a in _layer_vals:
        deltas.append(sigmoid_delta(a))
    loss_d = loss
    for d, w in zip(reversed(deltas), reversed(_weights)):
        dd = loss * d
        loss_d = torch.mm(d, w.t())
        ds.append(dd)
    learning_rate = 0.1
    _as = copy.deepcopy(_layer_vals)
    _as.insert(0,x)
    del _as[-1]
#     print("\n Layer vals \n", _layer_vals)
    new_weights = []
    new_biases = []
    print("lengths", len(ds), len(_weights), len(_as))
    for d, w, a, b in zip(ds, reversed(_weights), _as, reversed(_biases)):
        wt = torch.mm(a.t(),d) * learning_rate
        new_weights.insert(0, wt)
        
        bi = b + d.sum()*learning_rate
        new_biases.insert(0,bi)
    return new_weights,  new_biases   

In [6]:
def train(_data, _labels, _nodes, _epochs):
    #param initialization
    weights = []
    for i in range(len(_nodes)-1):
        w = torch.randn(_nodes[i], _nodes[i+1])
        weights.append(w)
    biases = []
    for i in range(len(nodes)-1):
        b = torch.randn(1, nodes[i+1])
        biases.append(b)
    
    print("WEIGHTS ARRAY:\n", weights)
    print("BIASES ARRAY:\n", biases)

    print("\n TRAINING \n")
    y_out = forward_prop(_data, _nodes, weights, biases)
    weights_2, biases_2 = backprop_and_update(y_out, weights, biases, _data)
    print("Final Weights: \n", weights_2)
    print("Final Biases: \n", biases_2)
    
   

In [7]:
## initialize tensor for inputs, and outputs 
x = torch.randn((1, 5))
y = torch.randn((1, 1))
nodes = [5, 3, 2, 1]
train(x, y, nodes, 10)

WEIGHTS ARRAY:
 [tensor([[-0.8998,  0.6309, -0.3725],
        [-1.0160, -0.5756, -0.1092],
        [ 1.2034,  0.2349,  1.2789],
        [ 0.7998, -0.5387,  0.9961],
        [ 1.2737,  0.7636,  0.3684]]), tensor([[-1.1102,  2.2419],
        [-0.1650,  1.3534],
        [ 1.1400, -0.5284]]), tensor([[1.0533],
        [0.3705]])]
BIASES ARRAY:
 [tensor([[ 0.3554,  0.3254, -0.8494]]), tensor([[-2.3005, -0.8894]]), tensor([[-0.3711]])]

 TRAINING 

lengths 3 3 3
Final Weights: 
 [tensor([[-1.1972e-04, -2.6543e-04, -9.2815e-05],
        [-8.0556e-04, -1.7860e-03, -6.2453e-04]]), tensor([[-1.2523e-04, -4.1916e-04],
        [-6.3953e-04, -2.1406e-03],
        [-9.3846e-05, -3.1412e-04]]), tensor([[-0.0017],
        [-0.0017],
        [ 0.0006],
        [ 0.0030],
        [ 0.0013]])]
Final Biases: 
 [tensor([[ 0.3495,  0.3195, -0.8553]]), tensor([[-2.3050, -0.8939]]), tensor([[-0.3746]])]
